In [1]:
import pandas as pd
import numpy as np
import os
import trimesh
import h5py
from meshparty import trimesh_io, trimesh_vtk, skeleton, utils
import vtk
import matplotlib.pyplot as plt
import datetime

In [2]:
mesh_dir = 'data/cell_meshes/' # or change to your desired folder
seg_source = 'graphene://https://minnie.microns-daf.com/segmentation/table/minnie65_public' #"precomputed://gs://iarpa_microns/minnie/minnie65/seg"
mm = trimesh_io.MeshMeta(cv_path=seg_source,
                         disk_cache_path=mesh_dir, 
                         cache_size=20)

In [3]:
# Load cellids into a list
astro_list = []
neuro_list = [864691135397985569]
vasc_list = []

print(f'number of astros:', len(astro_list))
print(f'number of neurons:', len(neuro_list))
print(f'number of vascular:', len(vasc_list))

number of astros: 0
number of neurons: 1
number of vascular: 0


In [4]:
neuro_list

[864691135397985569]

In [5]:
# Function to load .h5 file and convert it to a trimesh object
def load_h5_mesh(file_path):
    with h5py.File(file_path, 'r') as h5_file:
        try:
            # Assuming the dataset contains vertices and faces
            vertices = h5_file['vertices'][:]
            faces = h5_file['faces'][:]
            return trimesh.Trimesh(vertices=vertices, faces=faces)
        except KeyError as e:
            print(f"KeyError: {e}. Ensure the .h5 file has 'vertices' and 'faces' datasets.")
            return None

# Combine all the lists into a dictionary with their respective prefixes
prefix_dict = {
    'astro_': astro_list,
    'neuro_': neuro_list,
    'vasc_': vasc_list
}

# Make a dictionary to hold the mesh file for each cell id in the lists
mesh_dictionary = {}
for prefix, cell_list in prefix_dict.items():
    for cellid in cell_list:
        mesh_file = os.path.join(mesh_dir, str(cellid) + '.h5')
        mesh = load_h5_mesh(mesh_file)
        if mesh is not None:
            mesh_dictionary[prefix + str(cellid)] = mesh
        else:
            print(f"Failed to load mesh for file: {mesh_file}")


In [6]:
len(mesh_dictionary)

1

In [7]:
# Opacity settings for each cell type
astro_opac = 0.025
neuro_opac = 0.65
vasc_opac = 0.15

# Define the opacity for each prefix
opacity_dict = {
    'astro_': astro_opac,
    'neuro_': neuro_opac,
    'vasc_': vasc_opac
}

# Creating mesh actors with opacity and random colors
mesh_actor = {}
centroids = []
for prefix, cell_list in prefix_dict.items():
    for cellid in cell_list:
        cell_key = prefix + str(cellid)
        if cell_key in mesh_dictionary:
            random_color = list(np.random.random(size=3))
            cell_opac = opacity_dict[prefix]
            mesh_actor[cell_key] = trimesh_vtk.mesh_actor(mesh_dictionary[cell_key], opacity=cell_opac, color=random_color)
            centroids.append(mesh_dictionary[cell_key].centroid)

locals().update(mesh_actor)

# Calculate global mean centroid
if centroids:
    global_mean_centroid = np.mean(centroids, axis=0)
else:
    global_mean_centroid = np.array([0, 0, 0])
    print("No centroids available for mean calculation.")

# Creating a camera object and defining the view
camera = trimesh_vtk.oriented_camera(global_mean_centroid, backoff=800)

# Render the actors, will open a pop-up Python window
trimesh_vtk.render_actors(mesh_actor.values(), camera=camera)


setting up renderer
done setting up
actors added
camera set
render done
finalizing..


<vtkmodules.vtkRenderingOpenGL2.vtkOpenGLRenderer(0x00000220BF13EBB0) at 0x00000220C25B5168>

In [8]:
# save image to disk

save_dir = 'vtk_images/'
base_name = 'minnie_astro_neuro_vasc'
extension = 'png'

date_time_stamp = datetime.datetime.now().strftime("%Y_%m_%d_%H%M_%S")
filename = f"{save_dir}{base_name}_{date_time_stamp}.{extension}"

# uncomment to save

# trimesh_vtk.render_actors(mesh_actor.values(),
#                           filename=filename,
#                           do_save=True,
#                           scale=6,
#                           camera=camera
#                          )

# Load synapse tables
For first root_id in the neuro_list

In [9]:
table_path = 'data/synapse_tables/'

In [10]:
neuro_list

[864691135397985569]

In [11]:
syn_df_pre = pd.read_csv(f'{table_path}{neuro_list[0]}_syn_df_pre.csv')

# Convert 'ctr_pt_position' column to a list of int64 arrays
pre_ctr_pt_position_array = syn_df_pre['ctr_pt_position'].apply(
    lambda x: [np.int64(v) for v in x.strip('[]').split()]
).to_list()

# Convert the resulting list of lists into a numpy array
pre_ctr_pt_position_array = np.array(pre_ctr_pt_position_array, dtype=np.int64)

# Optional: Display the resulting array to verify
pre_ctr_pt_position_array

array([[325038, 129840,  21959],
       [311136, 100264,  18750],
       [325540, 104276,  20435],
       [311908, 122628,  17770],
       [338968, 134860,  20569],
       [313257,  88803,  19636],
       [333312, 117684,  17359],
       [331426, 116836,  17350],
       [325822, 124154,  20942],
       [324854, 129190,  22011],
       [338150, 110612,  21047],
       [327032, 129678,  22228],
       [327084, 129933,  22237],
       [330844, 118698,  23415],
       [336388, 105330,  17334],
       [318338, 127541,  21976],
       [328034, 128067,  22099],
       [308547, 116828,  19282],
       [330032, 115148,  22666],
       [339790, 109928,  20697],
       [318594, 128232,  22002],
       [331648, 116516,  23358],
       [327766, 126846,  21762],
       [318428, 122712,  21553],
       [329575, 128775,  22217],
       [316996, 124004,  20251],
       [316942, 122632,  20206],
       [316582, 126212,  20362],
       [338012,  92370,  21832],
       [316606, 125168,  20353],
       [33

In [12]:
syn_df_post = pd.read_csv(f'{table_path}{neuro_list[0]}_syn_df_post.csv')

# Convert 'ctr_pt_position' column to a list of int64 arrays
post_ctr_pt_position_array = syn_df_post['ctr_pt_position'].apply(
    lambda x: [np.int64(v) for v in x.strip('[]').split()]
).to_list()

# Convert the resulting list of lists into a numpy array
post_ctr_pt_position_array = np.array(post_ctr_pt_position_array, dtype=np.int64)

# Optional: Display the resulting array to verify
post_ctr_pt_position_array

array([[324478,  87826,  19945],
       [332959,  84214,  20383],
       [334106, 110182,  19205],
       ...,
       [319366,  97322,  20049],
       [318682, 111620,  20192],
       [325345, 104598,  20446]], dtype=int64)

In [13]:
# Multiplying each element in the array by [4, 4, 40]
scaling_factors = np.array([4, 4, 40])
transformed_presyn_xyz = pre_ctr_pt_position_array * scaling_factors
transformed_postsyn_xyz = post_ctr_pt_position_array * scaling_factors

# Print the result to verify
print(transformed_presyn_xyz)
print()
print(transformed_postsyn_xyz)


[[1300152  519360  878360]
 [1244544  401056  750000]
 [1302160  417104  817400]
 [1247632  490512  710800]
 [1355872  539440  822760]
 [1253028  355212  785440]
 [1333248  470736  694360]
 [1325704  467344  694000]
 [1303288  496616  837680]
 [1299416  516760  880440]
 [1352600  442448  841880]
 [1308128  518712  889120]
 [1308336  519732  889480]
 [1323376  474792  936600]
 [1345552  421320  693360]
 [1273352  510164  879040]
 [1312136  512268  883960]
 [1234188  467312  771280]
 [1320128  460592  906640]
 [1359160  439712  827880]
 [1274376  512928  880080]
 [1326592  466064  934320]
 [1311064  507384  870480]
 [1273712  490848  862120]
 [1318300  515100  888680]
 [1267984  496016  810040]
 [1267768  490528  808240]
 [1266328  504848  814480]
 [1352048  369480  873280]
 [1266424  500672  814120]
 [1337944  474728  643840]
 [1338444  504196  856160]
 [1338640  466912  645280]
 [1337964  492472  853240]
 [1294336  468200  818840]
 [1338740  462036  643840]
 [1348632  536672  828560]
 

In [14]:
# settings to visualize pre and post synaptic sites (as dots)
pre_color = (0.2, 0.9, 0.2)
post_color = (0.9, 0.2, 0.2)
pre_opac = 0.75
post_opac = 0.75
pre_size = 1000
post_size = 250
presyn_actor = trimesh_vtk.point_cloud_actor(transformed_presyn_xyz, size=pre_size, opacity=pre_opac, color=pre_color)  
postsyn_actor = trimesh_vtk.point_cloud_actor(transformed_postsyn_xyz, size=post_size, opacity=post_opac, color=post_color)

# Opacity settings for each cell type
astro_opac = 0.15
neuro_opac = 0.25
vasc_opac = 0.05

# Define the opacity for each prefix
opacity_dict = {
    'astro_': astro_opac,
    'neuro_': neuro_opac,
    'vasc_': vasc_opac
}

# Creating mesh actors with opacity and random colors
mesh_actor = {}
centroids = []
for prefix, cell_list in prefix_dict.items():
    for cellid in cell_list:
        cell_key = prefix + str(cellid)
        if cell_key in mesh_dictionary:
            random_color = list(np.random.random(size=3))
            cell_opac = opacity_dict[prefix]
            mesh_actor[cell_key] = trimesh_vtk.mesh_actor(mesh_dictionary[cell_key], opacity=cell_opac, color=random_color)
            centroids.append(mesh_dictionary[cell_key].centroid)

locals().update(mesh_actor)

# add pre and post synaptic sites to the mesh actor
mesh_actor['presyn_actor'] = presyn_actor
mesh_actor['postsyn_actor'] = postsyn_actor

locals().update(mesh_actor)

# Calculate global mean centroid
if centroids:
    global_mean_centroid = np.mean(centroids, axis=0)
else:
    global_mean_centroid = np.array([0, 0, 0])
    print("No centroids available for mean calculation.")

# Creating a camera object and defining the view
camera = trimesh_vtk.oriented_camera(global_mean_centroid, backoff=600)

# Render the actors, will open a pop-up Python window
trimesh_vtk.render_actors(mesh_actor.values(), camera=camera)



setting up renderer
done setting up
actors added
camera set
render done
finalizing..


<vtkmodules.vtkRenderingOpenGL2.vtkOpenGLRenderer(0x00000220C4C78370) at 0x00000220C25B5C48>

In [15]:
# save image to disk

save_dir = 'vtk_images/'
base_name = 'minnie_chandelier'
extension = 'png'

date_time_stamp = datetime.datetime.now().strftime("%Y_%m_%d_%H%M_%S")
filename = f"{save_dir}{base_name}_{date_time_stamp}_syn_{neuro_list[0]}.{extension}"

# uncomment to save

# trimesh_vtk.render_actors(mesh_actor.values(),
#                           filename=filename,
#                           do_save=True,
#                           scale=6,
#                           camera=camera
#                          )